In [ ]:
from dotenv import load_dotenv

load_dotenv()
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
from vesuvius_challenge_rnd.data import Fragment, Scroll, MonsterSegmentRecto, MonsterSegmentVerso

## Fragments

### Load data

In [ ]:
fragments = [Fragment(i + 1) for i in range(3)]
fragments

### Visualize fragments

In [ ]:
fig, ax = plt.subplots(len(fragments), 4, figsize=(12, 10), sharex="row", sharey="row")
for i, axis in enumerate(ax):
    ax1, ax2, ax3, ax4 = axis

    # Set title for first row only.
    if i == 0:
        ax1.set_title("IR image")
        ax2.set_title("Papyrus mask")

        ax3.set_title("Ink labels")
        ax4.set_title("Slice 32 micro-CT")

    ir_img = fragments[i].load_ir_img()
    ax1.imshow(ir_img, cmap="gray")

    mask = fragments[i].load_mask()
    ax2.imshow(mask, cmap="binary")

    ink_labels = fragments[i].load_ink_labels()
    ax3.imshow(ink_labels, cmap="binary")

    subvolume = fragments[i].load_volume_as_memmap(31, 32)
    ax4.imshow(subvolume[0], cmap="gray")

plt.tight_layout()

plt.show()

## Scrolls

### Load data

In [ ]:
scrolls = [Scroll(i + 1) for i in range(2)]
scrolls

Let's see how many missing segments there are.

In [ ]:
for scroll in scrolls:
    print(f"Scroll {scroll.scroll_id} num missing segments: {scroll.n_missing_segments}")

### View shape distribution
Let's look at the shape distribution. First of all, do they all have 65 slices?

In [ ]:
unique_num_slices = set()
for scroll in scrolls:
    for segment in scroll.segments:
        unique_num_slices.add(segment.n_slices)
print(f"Unique number of slices: {unique_num_slices}")

So they all have 65 slices. How about the surface shapes?

In [ ]:
surface_shapes = []
labels = []
for scroll in scrolls:
    for segment in scroll.segments:
        surface_shapes.append(segment.surface_shape)
        labels.append(scroll.scroll_id)

surface_shape_df = pd.DataFrame({"Surface shape": surface_shapes, "Scroll": labels})
surface_shape_df[["Num rows", "Num columns"]] = surface_shape_df["Surface shape"].apply(pd.Series)
surface_shape_df = surface_shape_df.drop("Surface shape", axis=1)

joint_plot = sns.jointplot(
    data=surface_shape_df, x="Num rows", y="Num columns", hue="Scroll", kind="scatter"
)
joint_plot.fig.suptitle("Surface shape distribution by scroll", y=1.02)

plt.show()

### Area distribution

Let's look at the scroll segment area distribution.

In [ ]:
areas = []
labels = []
for scroll in scrolls:
    for segment in scroll.segments:
        try:
            area = segment.area_cm2
        except FileNotFoundError as e:
            print(f"Skipping segment {segment.segment_name}: {e}")
            continue
        areas.append(area)
        labels.append(scroll.scroll_id)
area_df = pd.DataFrame({"Area": areas, "Scroll": labels})
sns.histplot(data=area_df, x="Area", hue="Scroll")

plt.title("Area distribution by scroll")
plt.xlabel(r"Segment area $(cm^2)$")
plt.show()

### Author distribution

Here, we look at the distribution of authors (annotators) for each scroll segment.

In [ ]:
authors = []
labels = []
for scroll in scrolls:
    for segment in scroll.segments:
        try:
            author = segment.author
        except FileNotFoundError as e:
            print(f"Skipping segment {segment.segment_name}: {e}")
            continue
        authors.append(segment.author)
        labels.append(scroll.scroll_id)

author_df = pd.DataFrame({"Author": authors, "Scroll": labels})

sns.histplot(
    data=author_df,
    x="Author",
    hue="Scroll",
    element="step",
    stat="count",
    binwidth=0.5,
    discrete=True,
    alpha=0.5,
)

plt.title("Author distribution by scroll")
plt.xticks(rotation=30)
plt.show()

### Visualize a scroll segment

We look at about 10 slices from the first segment of the first scroll.

In [ ]:
z_min = 27
z_max = 37
n_slices_to_show = z_max - z_min
segment = scrolls[0][0]
print(segment)
fig, ax = plt.subplots(1, 1 + n_slices_to_show, figsize=(25, 25), sharex="row", sharey="row")

# Show mask.
ax1 = ax[0]
mask = segment.load_mask()
ax1.imshow(mask, cmap="binary")
ax1.set_title("Papyrus mask")

# Show subvolume.
subvolume = segment.load_volume_as_memmap(z_min, z_max)
for i, axis in enumerate(ax[1:]):
    axis.imshow(subvolume[i], cmap="gray")
    axis.set_title(f"Slice {i + z_min}", fontsize=10)

plt.tight_layout()

plt.show()

### Visualize the monster segment

These are already included in scroll 1, but we can also create them separately.

#### Recto

In [ ]:
z_min = 31
z_max = 34
n_slices_to_show = z_max - z_min
segment = MonsterSegmentRecto()
print(f"orientation: '{segment.orientation}'")
print(segment)
fig, ax = plt.subplots(1, 1 + n_slices_to_show, figsize=(25, 25), sharex="row", sharey="row")

# Show mask.
ax1 = ax[0]
mask = segment.load_mask()
ax1.imshow(mask, cmap="binary")
ax1.set_title("Papyrus mask")

# Show subvolume.
subvolume = segment.load_volume_as_memmap(z_min, z_max)
for i, axis in enumerate(ax[1:]):
    axis.imshow(subvolume[i], cmap="gray")
    axis.set_title(f"Slice {i + z_min}", fontsize=10)

plt.tight_layout()

plt.show()

#### Verso

In [ ]:
z_min = 31
z_max = 34
n_slices_to_show = z_max - z_min
segment = MonsterSegmentVerso()
print(f"orientation: '{segment.orientation}'")
print(segment)
fig, ax = plt.subplots(1, 1 + n_slices_to_show, figsize=(25, 25), sharex="row", sharey="row")

# Show mask.
ax1 = ax[0]
mask = segment.load_mask()
ax1.imshow(mask, cmap="binary")
ax1.set_title("Papyrus mask")

# Show subvolume.
subvolume = segment.load_volume_as_memmap(z_min, z_max)
for i, axis in enumerate(ax[1:]):
    axis.imshow(subvolume[i], cmap="gray")
    axis.set_title(f"Slice {i + z_min}", fontsize=10)

plt.tight_layout()

plt.show()